In [89]:
import sqlite3
import pandas as pd
import numpy as np
from random import choice
from sql import *


Adding a single new Artist:

In [39]:
db = Chinook("chinook.db")
aId = db.addArtist(name="The Singer")
aId

1

In [40]:
db.getArtist(aId) # this is the artist added

Artist(ArtistId='1', Name='The Singer')

Adding more artists:

In [41]:
firstNames = ["John", "Johan", "Jan", "Ivan"]
lastNames = ["Smith", "Kowalski", "Kovalsky"]

names = [0, 0, 0, 0, 0]
for i in range(5):
    n = choice(firstNames) + " " + choice(lastNames)
    names[i] = n
    db.addArtist(name=n)

In [42]:
for a in db.allArtists()[-8:-1]:
    print(a)

Artist(ArtistId='1', Name='The Singer')
Artist(ArtistId='2', Name='Jan Kovalsky')
Artist(ArtistId='3', Name='Ivan Smith')
Artist(ArtistId='4', Name='Johan Kovalsky')
Artist(ArtistId='5', Name='Ivan Kowalski')


It is possible to access directly to the Name and Artist Id of such artists

In [43]:
for a in db.allArtists()[-8:-1]:
    print(a.ArtistId)
    print(a.Name)


1
The Singer
2
Jan Kovalsky
3
Ivan Smith
4
Johan Kovalsky
5
Ivan Kowalski


Quering directly on the db file and converting it to pandas dataframe:

In [44]:
q = '''
    SELECT
        t.TrackId, t.Name AS TrackName, t.AlbumId,
        a.Title AS AlbumTitle, 
        ar.Name AS ArtistName

    FROM tracks t 
    INNER JOIN albums a ON t.AlbumId = a.AlbumId
    INNER JOIN artists ar ON a.ArtistId = ar.ArtistId
'''

con = sqlite3.connect("chinook.db")  # enable the connection with the database
df = pd.read_sql_query(q, con)  # apply the query and import in Pandas
df.head()

,TrackId,TrackName,AlbumId,AlbumTitle,ArtistName


In [45]:
df.TrackName[df.AlbumTitle == "For Those About To Rock We Salute You"]

Series([], Name: TrackName, dtype: object)

In [46]:
df.columns

Index(['TrackId', 'TrackName', 'AlbumId', 'AlbumTitle', 'ArtistName'], dtype='object')

In [47]:
df[df.AlbumTitle == "For Those About To Rock We Salute You"][["TrackName", "ArtistName", "AlbumTitle"]]

,TrackName,ArtistName,AlbumTitle


Importing new data from a CSV file:

In [90]:
df_new = pd.read_csv("https://raw.githubusercontent.com/mahkaila/songnames/master/SongCSV.csv")
df_new.head()

,SongNumber,SongID,AlbumID,AlbumName,ArtistID,ArtistLatitude,ArtistLocation,ArtistLongitude,ArtistName,Danceability,Duration,KeySignature,KeySignatureConfidence,Tempo,TimeSignature,TimeSignatureConfidence,Title,Year
0,1,b'SOVLGJY12A8C13FBED',223563,b'Call of the Mastodon',b'ARMQHX71187B9890D3',NaN,b'Atlanta GA',NaN,b'Mastodon',0.0,280.21506,5,0.555,173.205,5,0.120,b'Deep Sea Creature',2001
1,2,b'SOMZWCG12A8C13C480',300848,b'Fear Itself',b'ARD7TVE1187B99BFB1',NaN,b'California - LA',NaN,b'Casual',0.0,218.93179,1,0.736,92.198,4,0.778,"bI Didn't Mean To""""",0
2,3,b'SOCIWDW12A8C13D406',300822,b'Dimensions',b'ARMJAGH1187FB546F3',35.14968,b'Memphis TN',-90.04892,b'The Box Tops',0.0,148.03546,6,0.169,121.274,4,0.384,b'Soul Deep',1969
3,4,b'SOXVLOJ12AB0189215',514953,b'Las Numero 1 De La Sonora Santanera',b'ARKRRTF1187B9984DA',NaN,b'',NaN,b'Sonora Santanera',0.0,177.47546,8,0.643,100.070,1,0.000,b'Amor De Cabaret',0
4,5,b'SONHOTT12A8C13493C',287650,b'Friend Or Foe',b'AR7G5I41187FB4CE6C',NaN,b'London England',NaN,b'Adam Ant',0.0,233.40363,0,0.751,119.293,4,0.000,b'Something Girls',1982


In [91]:
# add Artist method:
# ArtistId FK added automatically
df_new["ArtistName"] = df_new["ArtistName"].str[2:-1] # Name in artists table

# add Album 
# AlbumId FK added automatically
# ArtistId FK as input
df_new["AlbumName"] = df_new["AlbumName"].str[2:-1]  # Title in albums table

# add Track
# TrackId FK added automatically
# AlbumId FK as input
df_new["Title"] = df_new["Title"].str[2:-1]  # Title in tracks table

df_new = df_new[["ArtistName", "AlbumName", "Title"]]
df_new.head()

,ArtistName,AlbumName,Title
0,Mastodon,Call of the Mastodon,Deep Sea Creature
1,Casual,Fear Itself,"Didn't Mean To"""
2,The Box Tops,Dimensions,Soul Deep
3,Sonora Santanera,Las Numero 1 De La Sonora Santanera,Amor De Cabaret
4,Adam Ant,Friend Or Foe,Something Girls


In [92]:
df_new.tail()

,ArtistName,AlbumName,Title
9996,Moonspell,Sin / Pecado,The Hanged Man
9997,Danny Williams,Collection,The Wonderful World Of The Young
9998,Winston Reedy,Reality,Sentimental Man
9999,"Myrick Freeze"" Guillory'",Nouveau Zydeco,Zydeco In D-Minor
10000,Seventh Day Slumber,Once Upon A Shattered Life,Shattered Life


In [51]:
df_new[df_new["ArtistName"]  == "Moonspell"]

,ArtistName,AlbumName,Title
1484,Moonspell,The Butterfly Effect,Tired
3928,Moonspell,Second Skin,Sacred
5989,Moonspell,Wolfheart (re-issue + Bonus Tracks),Alma Mater
6245,Moonspell,Wolfheart (re-issue + Bonus Tracks),Trebraruna
9996,Moonspell,Sin / Pecado,The Hanged Man


In [52]:
db.addArtist(name = "Moonspell")

7

In [53]:
db.getArtist(artistId = 7)

Artist(ArtistId='7', Name='Moonspell')

In [54]:
db.addAlbum(title = "Wolfheart (re-issue + Bonus Tracks)", artistId = 7)

1

In [55]:
db.addTrack(name= "Alma Mater", albumId = 1)

1

In [56]:
db.addTrack(name = "Trebraruna", albumId = 1)

2

In this part, I am able to put on the database all the data from the new_dataset! 

It would be important to avoid using inner loops in order to 

In [96]:
db = Chinook("chinook.db")

In [8]:
import sqlalchemy
sqlalchemy.__version__

'1.4.32'

In [97]:
artists = df_new["ArtistName"].unique()

for artist in artists:
    artistId = db.addArtist(name = artist) # add all unique artists to the database
    albums = df_new[df_new["ArtistName"] == artist]["AlbumName"].unique()
    
    for album in albums:
        albumId = db.addAlbum(title = album, artistId = artistId)
        tracks = df_new[(df_new["ArtistName"] == artist) & (df_new["AlbumName"] == album)]["Title"].unique()
        
        for track in tracks:
            trackId = db.addTrack(name = track, albumId = albumId)

In [146]:
df_artists = pd.DataFrame()
df_artists["artists"] = df_new["ArtistName"].unique()
df_artists["Id"] = np.arange(1, len(df_artists["artists"])+1, 1)
df_artists.head()

df_albums = pd.DataFrame()
df_albums["albums"] = df_new["AlbumName"].unique()
df_albums["artistId"] = None # find a way to put the artistId

df_tracks = pd.DataFrame()
df_tracks["tracks"] = df_new["Title"].unique()
df_tracks["albumId"] = None # find a way to puy albumId

tot_albums = np.array(None)
tot_tracks = np.array(None)
for artist in artists:
    albums = df_new[df_new["ArtistName"] == artist]["AlbumName"].unique()
    np.append([tot_albums], [albums])
    
    for album in albums:
        tracks = df_new[(df_new["ArtistName"] == artist) & (df_new["AlbumName"] == album)]["Title"].unique()
        np.append([tot_tracks], [tracks])

#print(len(tot_albums), len(tot_tracks))  
tot_albums


array(None, dtype=object)

In [140]:
df_new.columns

Index(['ArtistName', 'AlbumName', 'Title'], dtype='object')

In [134]:
by_artist = df_new.groupby("ArtistName").sum()
by_album = df_new.groupby("AlbumName").sum()
by_track = df_new.groupby("Title").sum()


Accessing to all the albums, with each element being a class with AlbumId, Title, ArtistId  

In [115]:
albums = db.allAlbums()
album = albums[3] 
#album.tracks

DetachedInstanceError: Parent instance <Album at 0x148f7bb80> is not bound to a Session; lazy load operation of attribute 'tracks' cannot proceed (Background on this error at: https://sqlalche.me/e/14/bhk3)

In [113]:
tracks = db.allTracks()
track = tracks[3]
print(np.array([track.TrackId, track.Name, track.AlbumId]))

['4' 'Megalodon' '3']


In [88]:
artist = "Mastodon"

album = "Crack The Skye"
df_new[(df_new["ArtistName"] == artist) & (df_new["AlbumName"] == album)]["Title"].unique()

array(['The Czar: Usurper/Escape/Martyr/Spiral (Album Version)'],
      dtype=object)